In [35]:
import pandas as pd
import glob
import requests
import json
import time

# Project notes

- Identify transactions that are missing from OC that are on any of the accounts (meaning banks, stripe, wise, paypal)
- Identify any transactions on the platform that are missing from the accounts
- Prep a balance/p&l with minimal wrangling

Project steps:

- Clarify project intention and goal (fill in project assessment in Make OC)
- Map out data 
- Write scripts 
- Analyze and prepare accounts

# Load config

In [36]:
# host variables
host = 'opensource'
dateFrom = '2022-12-31T22:59:59.999Z'
dateTo = '2023-12-31T22:59:59.999Z'
year = '2023'

In [37]:
# load API tokens from .env file
import os
from dotenv import load_dotenv
load_dotenv()
octoken = os.getenv('OCTOKEN')
wisetoken = os.getenv('WISETOKEN')

# set the profile id of the target Wise profile
wiseprofile = os.getenv('WISEPROFILE')

In [38]:
# Reload the data or not - set these to false after the first run to avoid reloading
reloadOC = False
reloadWise = False
reloadStripe = True

# Get data

In [39]:
# load transactions from open collective or from local file

def fetch_transactions():
    url = f"https://opencollective.com/api/graphql/v2?personalToken={octoken}"

    # Your GraphQL query
    query = """
    query TransactionsTable(
        $hostAccount: AccountReferenceInput,
        $limit: Int!,
        $offset: Int!,
        $type: TransactionType,
        $paymentMethodType: [PaymentMethodType],
        $dateFrom: DateTime,
        $dateTo: DateTime,
        $kind: [TransactionKind],
        $includeIncognitoTransactions: Boolean,
        $includeGiftCardTransactions: Boolean,
        $includeChildrenTransactions: Boolean,
        $virtualCard: [VirtualCardReferenceInput],
        $orderBy: ChronologicalOrderInput,
        $group: String,
        $includeHost: Boolean,
        $expense: ExpenseReferenceInput,
        $order: OrderReferenceInput
      ) {
        transactions(
          host: $hostAccount
          limit: $limit
          offset: $offset
          type: $type
          paymentMethodType: $paymentMethodType
          dateFrom: $dateFrom
          dateTo: $dateTo
          kind: $kind
          includeIncognitoTransactions: $includeIncognitoTransactions
          includeGiftCardTransactions: $includeGiftCardTransactions
          includeChildrenTransactions: $includeChildrenTransactions
          includeDebts: true
          virtualCard: $virtualCard
          orderBy: $orderBy
          group: $group
          includeHost: $includeHost
          expense: $expense
          order: $order
        ) {
          ...TransactionsTableQueryCollectionFragment
          __typename
        }
      }
      
      fragment TransactionsTableQueryCollectionFragment on TransactionCollection {
        totalCount
        offset
        limit
        nodes {
          id
          uuid
          kind
          amount {
            currency
            valueInCents
            __typename
          }
          amountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          netAmount {
            currency
            valueInCents
            __typename
          }
          netAmountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          paymentProcessorFee {
            currency
            valueInCents
            __typename
          }
          paymentMethod {
            name
            service
            sourcePaymentMethod {
              id
              name
              service
              __typename
            }
            type
          }
          giftCardEmitterAccount {  
            id
            name
            slug
            __typename
          }
          group
          type
          description
          createdAt
          merchantId
          isRefunded
          isRefund
          refundTransaction {
            id
          }
          isOrderRejected
          account {
            ... on AccountWithParent {
              parent {
                id
                slug
                name
              }
            }
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          oppositeAccount {
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          expense {
            id
            type
            description
            invoiceInfo 
            tags
            payee {
              id
              name
              slug
              type
              __typename
            }
            __typename
          }
          permissions {
            id
            canRefund
            canDownloadInvoice
            canReject
            __typename
          }
          __typename
        }
        __typename
      }
    """

    headers = {
        "Authorization": f"Bearer {octoken}",
        "Content-Type": "application/json"
    }

    # Initial variables setup
    variables = {
        "hostAccount": {"slug": host},
        "includeIncognitoTransactions": True,
        "includeChildrenTransactions": True,
        "limit": 10000,  # Adjust if necessary but keep a sensible number to avoid server strain
        "offset": 0,  # Will be adjusted for each subsequent request
        "dateFrom": dateFrom,
        "dateTo": dateTo,
        "orderBy": {"field": "CREATED_AT", "direction": "DESC"},
        "includeHost": True
    }

    all_transactions = []  # To hold all transactions
    while True:
        
        # Try catch around json_data = response.json() to avoid errors

        max_retries = 10  # Setting the maximum number of retries
        retries = 0  # Initial retry count

        while retries < max_retries:
            try:
                # Make the HTTP request
                response = requests.post(
                    url, json={'query': query, 'variables': variables}, headers=headers)
                json_data = response.json()
                break 
            except:
                print("Request failed. Attempt:", retries + 1)
                print(response)
                retries += 1 
                time.sleep(10)

        if retries == max_retries:
            print("Maximum retry attempts reached. Exiting.")
        # Extract data
        transactions = json_data['data']['transactions']['nodes']
        all_transactions.extend(transactions)

        # Pagination: Update offset
        variables['offset'] += variables['limit']

        # Check if all transactions are fetched
        if len(all_transactions) >= json_data['data']['transactions']['totalCount']:
            break
        
        # print progress
        print(f'Fetched {len(all_transactions)} transactions')
        # sleep for 10 seconds to avoid server strain
        time.sleep(5)

    return all_transactions

# if account has property parent, replace account with parent
def replace_account_with_parent(transaction):
  if 'parent' in transaction['account']:
    transaction['account'] = transaction['account']['parent']
  return transaction

# apply the function to all transactions
def post_process_transactions(all_transactions):
  return list(map(replace_account_with_parent, all_transactions))

if reloadOC:
  all_transactions = fetch_transactions()
  # dump all transactions to a json file
  post_process_transactions(all_transactions)
  with open(f'data/{host}/{year}/{host}_{year}_all_platform_transactions.json', 'w') as f:
      json.dump(all_transactions, f, indent=2)
  # convert the json file to a dataframe
  df_platform_transactions = pd.json_normalize(all_transactions)
  # save dataframe as pickle
  df_platform_transactions.to_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')
else:
  df_platform_transactions = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')


In [40]:
# load transactions from wise

# Define the base URL for the Wise API
BASE_URL = 'https://api.transferwise.com/v1/transfers'

def fetch_all_transfers(profile_id=None, status=None, source_currency=None, 
                        target_currency=None, created_date_start=None, 
                        created_date_end=None, limit=100):
    headers = {
        'Authorization': f'Bearer {wisetoken}'
    }
    offset = 0
    all_transfers = []

    while True:
        # Prepare query parameters
        query_params = {'limit': limit, 'offset': offset}
        if profile_id:
            query_params['profile'] = profile_id
        if status:
            query_params['status'] = status
        if source_currency:
            query_params['sourceCurrency'] = source_currency
        if target_currency:
            query_params['targetCurrency'] = target_currency
        if created_date_start:
            query_params['createdDateStart'] = created_date_start
        if created_date_end:
            query_params['createdDateEnd'] = created_date_end

        # Make the GET request to the Wise API
        response = requests.get(BASE_URL, headers=headers, params=query_params)
        data = response.json()

        print(data)

        # Break if there are no more results
        if not data:
            break

        for transfer in data:
            all_transfers.append({
                'id': transfer['id'],
                'user': transfer['user'],
                'targetAccount': transfer['targetAccount'],
                'sourceAccount': transfer['sourceAccount'],
                'quote': transfer['quote'],
                'quoteUuid': transfer['quoteUuid'],
                'status': transfer['status'],
                'reference': transfer['reference'],
                'rate': transfer['rate'],
                'created': transfer['created'],
                'business': transfer['business'],
                'transferRequest': transfer['transferRequest'],
                'details.reference': transfer.get('details', {}).get('reference'),
                'hasActiveIssues': transfer['hasActiveIssues'],
                'sourceCurrency': transfer['sourceCurrency'],
                'sourceValue': transfer['sourceValue'],
                'targetCurrency': transfer['targetCurrency'],
                'targetValue': transfer['targetValue'],
                'customerTransactionId': transfer['customerTransactionId']
            })

        # Update the offset for the next iteration
        offset += limit

    return all_transfers

if reloadWise:
    transfers = fetch_all_transfers(profile_id=wiseprofile, created_date_start=dateFrom, created_date_end=dateTo)

    # create transfers dataframe
    df_wise_transactions = pd.DataFrame()

    for transfer in transfers:
        # add to dataframe with concat
        df_wise_transactions = pd.concat([df_wise_transactions, pd.DataFrame([transfer])])

    # save dataframe as pickle
    df_wise_transactions.to_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

In [41]:
# load transactions from stripe export
if reloadStripe:
    df_stripe_in = pd.read_csv(f'data/{host}/{year}/{host}_{year}_stripe.csv')
    # ensure only relevant columns
    df_stripe_in = df_stripe_in[[
    'id',
    'Created date (UTC)',
    'Description',
    'Seller Message',
    'Amount',
    'Amount Refunded',
    'Currency',
    'Converted Amount',
    'Converted Amount Refunded',
    'Fee',
    'Taxes On Fee',
    'Converted Currency',
    'Mode',
    'Status',
    'Statement Descriptor',
    'Customer ID',
    'Customer Description',
    'Captured',
    'Card ID',
    'PaymentIntent ID',
    'Application Fee',
    'Application ID',
    'from (metadata)',
    'to (metadata)'
    ]]
    df_stripe_in.to_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')
    


/var/folders/qf/j616b9d53yn77zcpky42s5r40000gn/T/ipykernel_46479/1810662741.py:3: DtypeWarning: Columns (26,27,28,29,30,31,37,38,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stripe_in = pd.read_csv(f'data/{host}/{year}/{host}_{year}_stripe.csv')


In [42]:
# load open collective data
df_platform = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')

# load stripe data
df_stripe = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')

# load wise data
df_wise = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

# load bank data
dir = f'data/{host}/{year}/oscbank_1'
df_bank_1 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_1 = df_bank_1.dropna(axis=1, how='all')

dir = f'data/{host}/{year}/oscbank_2'
df_bank_2 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_2 = df_bank_2.dropna(axis=1, how='all')

# Post process data

In [43]:
# post process

# remove failed and pending transactions from Stripe
df_stripe = df_stripe[df_stripe['Status'] != 'Failed']
df_stripe = df_stripe[df_stripe['Status'] != 'Pending']

# remove cancelled from Wise
df_wise = df_wise[df_wise['status'] != 'cancelled']

# Inspect transactions dataframe

In [44]:
# get unique transaction kind
transaction_kinds = df_platform_transactions['kind'].unique()

# sample 2 transactions for each kind
df_sample = df_platform_transactions.groupby('kind').apply(lambda x: x.sample(2))

df_sample

id  \
kind                                                                         
ADDED_FUNDS                   596922  e531304d-d6e2-47ad-b5bd-ce86a08dbd7b   
                              779752  550f0a1b-b4e6-4882-b7ca-7745fb6f8fe1   
BALANCE_TRANSFER              582118  5917e9b9-62c8-4154-8531-710f514effb8   
                              582119  07e16540-fc03-4994-9f24-9b2cbc6fd2a9   
CONTRIBUTION                  712889  97db33f5-b234-4dfc-94e2-72bdd5e024e0   
                              435201  8483dc78-e625-4595-b9de-6edc92bbcebc   
EXPENSE                       390742  61b2b30b-6598-41b2-8914-fbc0675312f7   
                              719164  2196fa16-901b-4219-ab4f-80fc6133b08a   
HOST_FEE                      184575  7c0f6654-9f65-44b6-bd1e-23216d792430   
                              683338  c4c9c734-00af-4163-b710-3422814b44b9   
HOST_FEE_SHARE                90487   f9631266-5a72-4024-bb45-36f6f26f7c20   
                              678314  fe730b5a-331f-4fe6-a101-6e8b15c1fb19   
HOST_FEE_SHARE_DEBT           308022  32c2795d-b2a7-44e4-888f-c7b2501f69cc   
                              235917  552979e6-e316-4113-ac91-1a47ca665f03   
PAYMENT_PROCESSOR_COVER       87703   a3c151b7-5725-462a-8669-a3f908edb467   
                              81834   0f3bf403-448e-4df5-994b-7ffd5a398ab2   
PAYMENT_PROCESSOR_DISPUTE_FEE 789689  db49f233-d532-48a8-a990-db3dfcb33a4b   
                              583134  b9487b21-deed-4116-bb6e-c48df56e7409   
PLATFORM_TIP_DEBT             102056  a3ce72b4-3744-470a-aa07-b7879b3c078d   
                              665608  67c14665-63a5-4543-95df-738e595aec2f   
PREPAID_PAYMENT_METHOD        226465  b95d58fb-face-4b91-b6ac-472950c9bb14   
                              446104  d782668b-24a0-47eb-88b3-5ea7aa4714e8   

                                                                      uuid  \
kind                                                                         
ADDED_FUNDS                   596922  e531304d-d6e2-47ad-b5bd-ce86a08dbd7b   
                              779752  550f0a1b-b4e6-4882-b7ca-7745fb6f8fe1   
BALANCE_TRANSFER              582118  5917e9b9-62c8-4154-8531-710f514effb8   
                              582119  07e16540-fc03-4994-9f24-9b2cbc6fd2a9   
CONTRIBUTION                  712889  97db33f5-b234-4dfc-94e2-72bdd5e024e0   
                              435201  8483dc78-e625-4595-b9de-6edc92bbcebc   
EXPENSE                       390742  61b2b30b-6598-41b2-8914-fbc0675312f7   
                              719164  2196fa16-901b-4219-ab4f-80fc6133b08a   
HOST_FEE                      184575  7c0f6654-9f65-44b6-bd1e-23216d792430   
                              683338  c4c9c734-00af-4163-b710-3422814b44b9   
HOST_FEE_SHARE                90487   f9631266-5a72-4024-bb45-36f6f26f7c20   
                              678314  fe730b5a-331f-4fe6-a101-6e8b15c1fb19   
HOST_FEE_SHARE_DEBT           308022  32c2795d-b2a7-44e4-888f-c7b2501f69cc   
                              235917  552979e6-e316-4113-ac91-1a47ca665f03   
PAYMENT_PROCESSOR_COVER       87703   a3c151b7-5725-462a-8669-a3f908edb467   
                              81834   0f3bf403-448e-4df5-994b-7ffd5a398ab2   
PAYMENT_PROCESSOR_DISPUTE_FEE 789689  db49f233-d532-48a8-a990-db3dfcb33a4b   
                              583134  b9487b21-deed-4116-bb6e-c48df56e7409   
PLATFORM_TIP_DEBT             102056  a3ce72b4-3744-470a-aa07-b7879b3c078d   
                              665608  67c14665-63a5-4543-95df-738e595aec2f   
PREPAID_PAYMENT_METHOD        226465  b95d58fb-face-4b91-b6ac-472950c9bb14   
                              446104  d782668b-24a0-47eb-88b3-5ea7aa4714e8   

                                                               kind  \
kind                                                                  
ADDED_FUNDS                   596922                    ADDED_FUNDS   
                              779752                    ADDED_FUNDS   
BALANCE_TRANSFER              582118      

In [45]:
# show all columns
pd.set_option('display.max_columns', None)
df_platform.sample(10)

,id,uuid,kind,paymentMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
356520,f5b1e011-1964-4699-a33e-17381ad9dfed,f5b1e011-1964-4699-a33e-17381ad9dfed,HOST_FEE_SHARE,NaN,NaN,377f1bc8-66b5-4960-91f0-1de7e72bb2a0,DEBIT,Host Fee Share,2023-08-01T09:08:13.336Z,None,False,False,NaN,False,NaN,Debit,USD,-10,Amount,USD,-10,Amount,USD,-10,Amount,USD,-10,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3w-g860ej5l-avnodgx7,8686,Open Collective,opencollective,False,ORGANIZATION,Organization,8k03reyd-5agmq5v7-kog6lbwo-z7j4nxv9,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584814,f647f5da-1261-4c67-b6a1-ab9f1991f393,f647f5da-1261-4c67-b6a1-ab9f1991f393,HOST_FEE,NaN,NaN,76e0c14b-73a1-4443-8690-36efeb7de42d,CREDIT,Host Fee,2023-04-20T22:59:03.170Z,None,False,False,NaN,False,NaN,Credit,USD,40,Amount,USD,40,Amount,USD,40,Amount,USD,40,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,ggnxdwzj-3le5mpwe-zrvpy8rv-bko04a97,126137,.fmbot,fmbot,False,COLLECTIVE,Collective,jrkx5lmn-v904qjaj-0jmq8bwa-7zdygoe3,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137484,6b973ae0-676b-4821-8f78-1608fb88c5f2,6b973ae0-676b-4821-8f78-1608fb88c5f2,CONTRIBUTION,NaN,NaN,a54ddd46-2157-4335-a04e-6a7774374b7e,CREDIT,Monthly financial contribution to Eleventy (Ba...,2023-11-01T12:02:11.604Z,ch_3O7d7dBYycQg1OMf1oWy8A74,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,rvedj9wr-oz3a56dg-eea67blg-8x4m0ykn,37254.0,Eleventy,11ty,False,COLLECTIVE,Collective,mvrwng4k-j03dpbmb-ea0qz57o-yl9e8xba,449734,Oisín Quinn,oisinq,False,INDIVIDUAL,Individual,mlo94zn7-x08dpo5j-4vmqewga-3vjbrky5,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62331,50b286b9-7e37-4fe9-94b6-7474061bfd80,50b286b9-7e37-4fe9-94b6-7474061bfd80,HOST_FEE,NaN,NaN,41f7aef4-9a32-46d8-b6b0-61aa57f91362,CREDIT,Host Fee,2023-12-01T12:04:09.218Z,None,False,False,NaN,False,NaN,Credit,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,vrgbk357-l4x96ea9-g3apomew-a0jdyzn8,445129,PHP Foundation,phpfoundation,False,COLLECTIVE,Collective,ax8emk7l-nw54q78d-dyypgyvj-0o93zdrb,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [46]:
# count transactions by kind
df_platform.groupby(['kind']).size()

kind
ADDED_FUNDS                        2616
BALANCE_TRANSFER                    116
CONTRIBUTION                     193827
EXPENSE                            7582
HOST_FEE                         385894
HOST_FEE_SHARE                   192946
HOST_FEE_SHARE_DEBT               61479
PAYMENT_PROCESSOR_COVER             728
PAYMENT_PROCESSOR_DISPUTE_FEE        18
PLATFORM_TIP_DEBT                    11
PREPAID_PAYMENT_METHOD               13
dtype: int64

# Reconciling Stripe

## Automatic reconciliation by merchantId

In [47]:
# copy df_strip to df_s
df_stripe_reconciliation = df_stripe.copy()

In [48]:
df_stripe_reconciliation.head()

,id,Created date (UTC),Description,Seller Message,Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Taxes On Fee,Converted Currency,Mode,Status,Statement Descriptor,Customer ID,Customer Description,Captured,Card ID,PaymentIntent ID,Application Fee,Application ID,from (metadata),to (metadata)
0,ch_3OTXgHBYycQg1OMf0IfKN8Sw,2023-12-31 22:46:55,Monthly financial contribution to Logseq (Back...,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI7wZ2sLFYmMPi,https://opencollective.com/sanket-sharma,True,pm_1OTXmVBYycQg1OMfzXUwCvAm,pi_3OTXgHBYycQg1OMf0PkaNI3M,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/sanket-sharma,https://opencollective.com/logseq
3,ch_3OTXVMBYycQg1OMf0qVZ5tjm,2023-12-31 22:29:41,Financial contribution to OpenSCAD,Payment complete.,50.00,0.0,usd,50.00,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTXVpBYycQg1OMfien2j8Cr,pi_3OTXVMBYycQg1OMf02f4Vhyl,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/openscad
4,ch_3OTULSBYycQg1OMf10wlDceK,2023-12-31 19:06:47,Monthly financial contribution to LibreELEC,Payment complete.,10.00,0.0,usd,10.00,0.0,1.24,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI4UZkic5DVDO1,https://opencollective.com/daniel-dolejska,True,pm_1OTULSBYycQg1OMf7Lt0BwEi,pi_3OTULSBYycQg1OMf1saV7WWG,0.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/daniel-dolejska,https://opencollective.com/libreelec
5,ch_3OTUG8BYycQg1OMf0FRuZULZ,2023-12-31 19:03:18,Financial contribution to Spotube,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTUI5BYycQg1OMfAbkDmXhS,pi_3OTUG8BYycQg1OMf0AcGFa2r,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/spotube
7,ch_3OTT8EBYycQg1OMf0gR3PzrY,2023-12-31 17:49:54,Financial contribution to .fmbot (.fmbot lifet...,Payment complete.,49.99,0.0,usd,49.99,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTT94BYycQg1OMf9AOHyAVx,pi_3OTT8EBYycQg1OMf0xdn31JV,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/fmbot


In [49]:
# df_platform where kind is CONTRIBUTION and paymentMethod.service is STRIPE
df_platform_stripe = df_platform[(df_platform['kind'] == 'CONTRIBUTION') & (df_platform['paymentMethod.service'] == 'STRIPE')]

In [50]:
df_platform_stripe.head()

,id,uuid,kind,paymentMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
3,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,CONTRIBUTION,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Monthly financial contribution to Logseq (Back...,2023-12-31T22:46:58.403Z,ch_3OTXgHBYycQg1OMf0IfKN8Sw,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,n4gx0bro-v5m96nkd-3ay6d8lk-3ey7jzwa,744511,Sanket Sharma,sanket-sharma,False,INDIVIDUAL,Individual,8k03reyd-5agmq5r7-rlyplbwo-z7j4nxv9,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,f3fb4940-1026-4440-bd85-bc3c0cb439b0,f3fb4940-1026-4440-bd85-bc3c0cb439b0,CONTRIBUTION,NaN,NaN,9931763a-79b1-4141-923f-216db174f56e,CREDIT,Financial contribution to OpenSCAD,2023-12-31T22:29:44.179Z,ch_3OTXVMBYycQg1OMf0qVZ5tjm,False,False,NaN,False,NaN,Credit,USD,5000,Amount,USD,5000,Amount,USD,4825,Amount,USD,4825,Amount,USD,-175,Amount,rmvrwng4-kj03dpbk-ljzpz57o-yl9e8xba,132610.0,OpenSCAD,openscad,False,COLLECTIVE,Collective,n4gx0bro-v5m96n0l-878qd8lk-3ey7jzwa,467683,Holland Hopson,guest-8761eb54,False,INDIVIDUAL,Individual,ywz9j4av-god8pg8g-8ly6mr35-nxklb0e7,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,e438b64b-473b-405b-be1c-67c7cfc58fba,e438b64b-473b-405b-be1c-67c7cfc58fba,CONTRIBUTION,NaN,NaN,91d79b1b-e38d-4559-be03-ad4a5f5fa301,CREDIT,Monthly financial contribution to LibreELEC,2023-12-31T19:06:49.150Z,ch_3OTULSBYycQg1OMf10wlDceK,False,False,NaN,False,NaN,Credit,USD,1000,Amount,USD,1000,Amount,USD,926,Amount,USD,926,Amount,USD,-74,Amount,88rzownx-l9e50pxv-3vb6ymvb-dgk7j43a,204839.0,LibreELEC,libreelec,False,COLLECTIVE,Collective,e0adkjrv-8xwm69oy-zwl67z5n-4l93bgoy,744470,Daniel Dolejska,daniel-dolejska,False,INDIVIDUAL,Individual,eng0kzdy-vor4pzez-e7bpbma8-37xlw95j,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,8ee3d5b0-b031-4b06-b7ab-69a8419900c2,8ee3d5b0-b031-4b06-b7ab-69a8419900c2,CONTRIBUTION,NaN,NaN,0870251c-c506-421d-b969-9b98f303bb90,CREDIT,Financial contribution to Spotube,2023-12-31T19:03:20.581Z,ch_3OTUG8BYycQg1OMf0FRuZULZ,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,3kzxy4v0-7wlr6mvy-5m76mj9n-o8agdbe5,475166.0,Spotube,spotube,False,COLLECTIVE,Collective,zaxon793-jy8gpl0r-4k9pbrkd-emwl5v04,744469,Ignacho,guest-a4555b91,False,INDI

In [51]:
# for each row in df_stripe, find the corresponding row in df_platform, match df_stripe['id'] with df_platform['merchantId']
# if the row exists, add the platform transaction id to df_stripe in a new column called platform_transaction_id
# if the row does not exist, set the platform_transaction_id to None

# Merge the two dataframes on the 'id' from df_stripe and 'merchantId' from df_platform
merged_df = pd.merge(df_stripe_reconciliation, df_platform[['merchantId', 'id']], left_on='id', right_on='merchantId', how='left', suffixes=('', '_platform_transaction'))

# Drop the extra 'merchantId' column from the merge
merged_df.drop(columns='merchantId', inplace=True)

# Assign the resulting dataframe back to df_stripe
df_stripe_reconciliation = merged_df

In [52]:
# count the number of Stripe transactions that have a id_platform_transaction
print(f"Stripe transactions that have a corresponding platform transaction: {len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].notnull()])}")

# count the number of Stripe transactions that do not have a id_platform_transaction
print(f"Stripe transactions without a corresponding platform transaction: {len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].isnull()])}")

# print the percentage of Stripe transactions that have a corresponding platform transaction
print(f"Percentage of Stripe transactions that have a corresponding platform transaction: {round(len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].notnull()])/len(df_stripe_reconciliation)*100, 2)}%")

Stripe transactions that have a corresponding platform transaction: 131588
Stripe transactions without a corresponding platform transaction: 432
Percentage of Stripe transactions that have a corresponding platform transaction: 99.67%


## Reconciling remaining Stripe transactions

In [53]:
df_stripe_remainder = df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].isnull()]

In [54]:
df_stripe_remainder.head()

,id,Created date (UTC),Description,Seller Message,Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Taxes On Fee,Converted Currency,Mode,Status,Statement Descriptor,Customer ID,Customer Description,Captured,Card ID,PaymentIntent ID,Application Fee,Application ID,from (metadata),to (metadata),id_platform_transaction
116,ch_3OSjH1BYycQg1OMf1SrSebC3,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfAPu0eks5,pi_3OSjH1BYycQg1OMf1hLSOPbZ,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
117,ch_3OSjH1BYycQg1OMf03ruobeK,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfOEXlem38,pi_3OSjH1BYycQg1OMf0l0hNQLu,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
118,ch_3OSjH1BYycQg1OMf0fkywX7k,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMftpAiBEfm,pi_3OSjH1BYycQg1OMf0QvSjaGu,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
119,ch_3OSjH0BYycQg1OMf04HatfLy,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfqhqMfElP,pi_3OSjH0BYycQg1OMf0xkJN2s7,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
175,ch_3OSN05BYycQg1OMf1xfVvxkZ,2023-12-28 17:04:15,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSN0EBYycQg1OMf6jMM1Jzf,pi_3OSN05BYycQg1OMf13FUXY3w,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN


In [55]:
# count df_stripe_remainder by 'Status'
df_stripe_remainder.groupby(['Status']).size()

Status
Paid        143
Refunded    289
dtype: int64

In [56]:
# count df_stripe_remainder by 'Seller Message'
df_stripe_remainder.groupby(['Seller Message']).size()

Seller Message
One of your rules allowed this payment, overriding Stripe's risk evaluation and any address or CVC checks.     41
Payment complete.                                                                                             376
Stripe evaluated this payment as having elevated risk, and placed it in your manual review queue.              15
dtype: int64

In [57]:
# sum amount of df_stripe_remainder by status
df_stripe_remainder.groupby(['Status'])['Amount'].sum()

Status
Paid        3907.0
Refunded     293.0
Name: Amount, dtype: float64

In [61]:
# count df_stripe_remainder by 'Created date (UTC)' date only
# create a new column with the date only
df_stripe_remainder['Created date'] = pd.to_datetime(df_stripe_remainder['Created date (UTC)']).dt.date

df_stripe_remainder.groupby(['Created date']).size()

/var/folders/qf/j616b9d53yn77zcpky42s5r40000gn/T/ipykernel_46479/2206905382.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stripe_remainder['Created date'] = pd.to_datetime(df_stripe_remainder['Created date (UTC)']).dt.date


Created date
2023-01-25     1
2023-03-13    10
2023-03-14    15
2023-03-15     2
2023-03-16     7
              ..
2023-06-29     8
2023-06-30     2
2023-12-27     2
2023-12-28     5
2023-12-29     4
Length: 73, dtype: int64

In [58]:
# count df_stripe_remainder by 'to (metadata)'
df_stripe_remainder.groupby(['to (metadata)']).size()

to (metadata)
https://opencollective.com/advanced-bim-frontend-developer-course      1
https://opencollective.com/curl                                       37
https://opencollective.com/hledger                                     1
https://opencollective.com/manjaro-us                                 10
https://opencollective.com/mvvmcross                                   1
https://opencollective.com/openaddresses                             324
dtype: int64

In [59]:
df_platform.head()

,id,uuid,kind,paymentMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
0,a985a0d8-c94e-4173-9918-3246e3231021,a985a0d8-c94e-4173-9918-3246e3231021,HOST_FEE_SHARE,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,DEBIT,Host Fee Share,2023-12-31T22:46:58.361Z,None,False,False,NaN,False,NaN,Debit,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3w-g860ej5l-avnodgx7,8686,Open Collective,opencollective,False,ORGANIZATION,Organization,vedj9wro-z3a56dzb-zllq7blg-8x4m0ykn,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,377540a1-80e4-45dd-8cf8-44e5a8d04f4d,377540a1-80e4-45dd-8cf8-44e5a8d04f4d,HOST_FEE,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Host Fee,2023-12-31T22:46:58.361Z,None,False,False,NaN,False,NaN,Credit,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914,Logseq,logseq,False,COLLECTIVE,Collective,vrgbk357-l4x96ey9-yl4pomew-a0jdyzn8,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d8b98e5b-60a2-4bb0-8e66-a0528ecaaf7c,d8b98e5b-60a2-4bb0-8e66-a0528ecaaf7c,HOST_FEE,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,DEBIT,Host Fee,2023-12-31T22:46:58.352Z,None,False,False,NaN,False,NaN,Debit,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,0,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004,Open Source Collective,opensource,False,ORGANIZATION,Organization,a47byg9n-xozdp8n7-nlmpmjlv-03rek5w8,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,CONTRIBUTION,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Monthly financial contribution to Logseq (Back...,2023-12-31T22:46:58.403Z,ch_3OTXgHBYycQg1OMf0IfKN8Sw,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,n4gx0bro-v5m96nkd-3ay6d8lk-3ey7jzwa,744511,Sanket Sharma,sanket-sharma,False,INDIVIDUAL,Individual,8k03reyd-5agmq5r7-rlyplbwo-z7j4nxv9,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Reconciling Wise

# Reconciling Bank

# Reconciling Paypal